# Project 3 Shakespearean Sonnets RNN Part 6


In [46]:
### Preprocessing Choices

# Tokenizing your words
# Split up sequencies
# hypenated words
# capitalization?
# punctuation

# What changed?

# What didn't work/what changed 

# Analysis on the dataset?

In [46]:
import random
import os
import fileinput
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random

# Initial Processing
- I'm not actually changing anything about the given text I'm literally just tokenizing the input and then passing the training data into the LSTM model.
- I am making 5 char jumps between samples of 40
- I tried to use the Keras tokenizer but it wasn't recognizing all of the unique character so I wrote my own
- I also decided to remove new line characters because it ruined the format of the sonnet 
- I had a very high number of spaces, so I also removed white space from between the sonnets in the given text file and the numbers of the sonnets

In [47]:
# Turn text file into training set

# training data should be 40 character sequences from the sonnets
# take all possible subsequences of 40 consectives from the dataset
# use semi-redundant requences-> pick sequences starting every n-th character


def get_char_repr(char_to_index, char):
    unique_words = char_to_index.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[char_to_index[char]] = 1
    return feature_representation 

def preprocess_init(text):
    # Convert text to dataset using semi-redundant sequences
    unique_chars = sorted(list(set(text)))
    skip = 5 
    char_len = 40
    vocab_dict = {}
    for i in range(len(unique_chars)):
        vocab_dict[unique_chars[i]] = i
    
    #list of sets fo 40 characters
    sequences = [] 
    # individual final characters 
    characters = []
    
    # generate seqs of 40 chars by looping through whole thing
    for i in range(0, len(text)-40, skip):
        sequences.append(text[i:i+40]) # sequence
        characters.append(text[i+41]) # char 
    
    # need to reshape because LSTM is being moody and wants a 3D thing for x
    trainX = np.zeros((len(sequences), 40, len(unique_chars)))
    trainY = np.zeros((len(sequences), len(unique_chars)))
    # put 1s into the places where things fit the correct char
    for index in range(len(sequences)):
        for seq in range(len(sequences[0])):
            trainX[index, seq] = get_char_repr(vocab_dict, sequences[index][seq])
        trainY[index] = get_char_repr(vocab_dict, characters[index])
        
    return trainX,trainY, vocab_dict

In [48]:
text = ""
for line in fileinput.FileInput('data/shakespeare.txt'):
    if line.rstrip():
        text+=line.lstrip()
text = text.replace("\n","")
text = text.replace("1","")
text = text.replace("2","")
text = text.replace("3","")
text = text.replace("4","")
text = text.replace("5","")
text = text.replace("6","")
text = text.replace("7","")
text = text.replace("8","")
text = text.replace("9","")
text = text.lower()
seqX, chars, vocab_map = preprocess_init(text)
unique_chars = sorted(list(set(text)))
map_len = len(vocab_map)

In [49]:
print(chars.shape)
print(seqX.shape)

(18301, 38)
(18301, 40, 38)


In [50]:
# Train a character-based LSTM model

def train_rnn(trainX, characters):
    
    model = Sequential()
    # single layers of 100-200 LSTM units
    model.add(LSTM(200, input_shape=(40,map_len)))
    # fully connected dense output layer with a softmax nonlinearity
    model.add(Dense(map_len, activation='softmax'))

    # Train model to minimize categorical cross-entropy
    model.compile(optimizer = 'rmsprop', # unclear if we need this
                  loss="categorical_crossentropy",
                  metrics = ["accuracy"]    
                  # we want accuracy over 0.6 on training data
                 )

    # train for many epochs so loss converges
    print("Train model...")
    model.fit(trainX, characters, epochs = 30, batch_size = 32)
    return model
    

In [51]:
rnn_model = train_rnn(seqX, chars)

Train model...
Epoch 1/30
18301/18301 [==============================] - 31s 2ms/step - loss: 2.9021 - accuracy: 0.1847
Epoch 2/30
18301/18301 [==============================] - 34s 2ms/step - loss: 2.7349 - accuracy: 0.2181
Epoch 3/30
18301/18301 [==============================] - 29s 2ms/step - loss: 2.6564 - accuracy: 0.2383
Epoch 4/30
18301/18301 [==============================] - 28s 2ms/step - loss: 2.6034 - accuracy: 0.2498
Epoch 5/30
18301/18301 [==============================] - 28s 2ms/step - loss: 2.5568 - accuracy: 0.2622
Epoch 6/30
18301/18301 [==============================] - 28s 2ms/step - loss: 2.5113 - accuracy: 0.2740
Epoch 7/30
18301/18301 [==============================] - 28s 2ms/step - loss: 2.4625 - accuracy: 0.2833
Epoch 8/30
18301/18301 [==============================] - 28s 2ms/step - loss: 2.4068 - accuracy: 0.2958
Epoch 9/30
18301/18301 [==============================] - 28s 2ms/step - loss: 2.3434 - accuracy: 0.3106
Epoch 10/30
18301/18301 [===============

In [57]:
def generate_next_char(z_vector,temperature):
    # implement temp parameter myself as part of the poem generation algo
    # add a Lambda layer to LSTM during prediction or write a function
    z_vector = np.asarray(z_vector)
    num = np.exp(z_vector/temperature)
    denom = np.sum(np.exp(z_vector/temperature))
    index_predictions = num/denom
    max_prediction_index = np.argmax(index_predictions)
    
    # have to find the char that goes with the index
    char = ""
    for i in vocab_map.keys():
        if vocab_map[i] == max_prediction_index:
            char = i
    return char

In [58]:
### Poetry Generation 
# to generate poems, draw softmax samples from the trained model
def generate_sonnet_without_random_seed():
    generated_sonnet = ''
    seed = "shall i compare thee to a summer's day? " # first sentence 
    generated_sonnet += seed
    for line in range(13*40): # of there are 14 lines in a sonnet and our seed is the first
        # each line is about 40 characters long?
            
        features = np.zeros((1,40,map_len))
        for i in range(40):
            features[0,i] = get_char_repr(vocab_map, generated_sonnet[len(generated_sonnet)-40+i])
        
        # generate the next value 
        predictions = rnn_model.predict(features)[0]
        next_char = generate_next_char(predictions, 0.25) # change around the temp
        generated_sonnet += next_char 
        
    # separate poem by line and print out
    for i in range(13):
        print(generated_sonnet[i*40:(i+1)*40])
    return generated_sonnet


In [59]:
g = generate_sonnet_without_random_seed()

shall i compare thee to a summer's day? 
hvrmaee o tysepttsettlgfrgtwiciu o eoe f
et rnee  ns lvee o e erae.had hah hseni 
aa ty uddan ee ee hrietonmsapigta,nt frs
 tm evrue o a adyte ee hrmgindo o o o en
etrngtiltwigigat haefrngi o o onestaditi
t,ta oneo fnttrmgis i,o,onoh hswlss ifon
wlswa asatntnd nttad o hnettnssttt'gtfn 
wmne hae wih htiodih hswisot hah har hah
 hrwihihdd nweo ntgt,lfgtogtmnyubtft lvs
rcoie o al o evre.hnrswas hshvr i o adia
tnsdsem hreeint,o ad htetntadehb ngnud o
 hvit o one ha  iln e eeprettogt,hwrn ha


In [60]:
def generate_sonnet_random_seed():
    generated_sonnet = ''
    start = random.randint(1,len(text)-40)
    seed = text[start:start+40] # first sentence 
    generated_sonnet += seed
    for line in range(13*40): # of there are 14 lines in a sonnet and our seed is the first
        # each line is about 40 characters long?
            
        features = np.zeros((1,40,map_len))
        for i in range(40):
            features[0,i] = get_char_repr(vocab_map, generated_sonnet[len(generated_sonnet)-40+i])
        
        # generate the next value 
        predictions = rnn_model.predict(features)[0]
        next_char = generate_next_char(predictions, 0.25) # change around the temp
        generated_sonnet += next_char 
        
    # separate poem by line and print out
    for i in range(13):
        print(generated_sonnet[i*40:(i+1)*40])
    return generated_sonnet

In [61]:
g1 = generate_sonnet_random_seed()

tue only is their show,they live unwooed
w h eieton tat frm eeie o a i esstl''rtt
tageofac u u nngt  eeo o i i ene nwenlig
tigigeti ah oe osies,nw hrwes ha  ons i 
cilsan i o e etnyudfngtih hne o esee,ha 
ihsat t evde ottnwwtat hvretesin onha o 
hsieto otonwswtfrsoo hngttt ettie i o ad
etmnm i esiettrtatalie  fott,o o tys tyt
 efrmte i n e ecsntrn hag,hwen hae on og
e,ad frn hvrewrno otan oeenrsteigtad tae
eo fnr raa e hvr  isntrcloe o i a etan e
edb,hm hrds hah wensntogttig ta ifersstf
ltttig fa eoe hvi i,hni hsepis afil,tyso
